In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout, ZeroPadding2D, UpSampling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import backend as K

import math
import matplotlib.pyplot as plt

import pypianoroll
import numpy as np
from pypianoroll import Multitrack, Track

In [ ]:
# run params
run_id = '3000'
music_name = 'wgan/'

RUN_FOLDER = 'run/'
RUN_FOLDER += '_'.join([run_id, music_name])

# Number of timestept the slices Pianorolls should have (Needs to be dividable by 16)
pianrollLength = 128

store_folder = os.path.join(RUN_FOLDER, 'store')
data_folder = os.path.join('data', music_name)

if not os.path.exists(RUN_FOLDER):
    os.makedirs(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'store'))
    os.mkdir(os.path.join(RUN_FOLDER, 'output'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    


mode = 'build' # 'load' # 
# Clip Threshold for weight clipping should be in range [-0.01, 0.01]
clip_threshold = 0.01

In [ ]:
# Set Note bounds for faster training
lowestNotePossible = 20
highestNotePossible = 108
# possibleNotes mus be dividable by 4 else the Architekture needs to be changed
possibleNotes = highestNotePossible - lowestNotePossible

In [ ]:
# Save the numpyArray for further us
'''
reshaped = np.load('data/preprocessed/midi_p128_dn88.npy')
isNormalized = False

reshaped = np.load('data/preprocessed/midi_normalized_p128_dn88.npy')
isNormalized = True
'''

reshaped = np.load('data/preprocessed/midi_binarized_p128_dn88.npy')
isNormalized = True

# Creating the neural Network

In [ ]:
### THE discriminator
discriminator_input = Input(shape=(pianrollLength,possibleNotes,1), name='discriminator_input')

x = discriminator_input

x = Conv2D(filters = 64, kernel_size = (5,5), strides=2, padding = 'same', name = 'discriminator_conv_0')(x)
x = LeakyReLU()(x)
x = Dropout(0.4)(x)

x = Conv2D(filters = 64, kernel_size = (5,5), strides=2, padding = 'same', name = 'discriminator_conv_1')(x)
x = LeakyReLU()(x)
x = Dropout(0.4)(x)

x = Conv2D(filters = 128, kernel_size = (5,5), strides=2, padding = 'same', name = 'discriminator_conv_2')(x)
x = LeakyReLU()(x)
x = Dropout(0.2)(x)

x = Conv2D(filters = 128, kernel_size = (5,5), strides=1, padding = 'same', name = 'discriminator_conv_3')(x)
x = LeakyReLU()(x)
x = Dropout(0.2)(x)


x = Flatten()(x)

discriminator_output = Dense(1)(x)
discriminator = Model(discriminator_input, discriminator_output, name= 'discriminator')

In [ ]:
discriminator.summary()

In [ ]:
# Quelle: https://stackoverflow.com/a/45199301
# Save the summary to a file 
#from contextlib import redirect_stdout

#with open(os.path.join(store_folder, 'modelsummarydiscriminator.txt'), 'w') as f:
#    with redirect_stdout(f):
#        discriminator.summary()

In [ ]:
z_dim = 100

generator_input = Input(shape=(z_dim,), name='generator_input')
generator_initial_dense_layer_size = (int(pianrollLength/4),int(possibleNotes/4),8)

x = generator_input
x = Dense(np.prod(generator_initial_dense_layer_size))(x)
#x = BatchNormalization(momentum=0.9)(x)

x = LeakyReLU()(x)
x = Reshape(generator_initial_dense_layer_size)(x)

x = UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest')(x)
x = Conv2D(filters = 128, kernel_size = (5,5), padding='same', name = 'generator_conv_0')(x)
#x = BatchNormalization(momentum=0.9)(x)
x = LeakyReLU()(x)   

x = UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest')(x)
x = Conv2D(filters = 64, kernel_size = (5,5), padding='same', name = 'generator_conv_1')(x)
#x = BatchNormalization(momentum=0.9)(x)
x = LeakyReLU()(x)  

x = Conv2D(filters = 64, kernel_size = (5,5), padding = 'same', name = 'generator_conv_2')(x)
#x = BatchNormalization(momentum=0.9)(x)
x = LeakyReLU()(x)  

x = Conv2D(filters = 1, kernel_size = (5,5), padding = 'same')(x)        
x = Activation('sigmoid')(x)


generator_output = x
generator = Model(generator_input, generator_output, name='generator')

In [ ]:
generator.summary()

In [ ]:
# Quelle: https://stackoverflow.com/a/45199301
# Save the summary to a file 
#from contextlib import redirect_stdout

#with open(os.path.join(store_folder, 'modelsummarygenerator.txt'), 'w') as f:
#    with redirect_stdout(f):
#        generator.summary()

In [ ]:
def set_trainable(model, isTrainable):
    for layer in model.layers:
        layer.trainable = isTrainable

In [ ]:
# Wasserstein loss function (Forster S. 117)
def wasserstein(y_true, y_pred):
    return -K.mean(y_true * y_pred)

In [ ]:
### COMPILE DISCRIMINATOR
discriminator.compile(
optimizer=RMSprop(lr=0.00005)
, loss = wasserstein #,  metrics = ['accuracy']
)
        
### COMPILE THE FULL GAN
set_trainable(discriminator, False)

model_input = Input(shape=(z_dim,), name='model_input')
model_output = discriminator(generator(model_input))
GANModel = Model(model_input, model_output)

opti = RMSprop(learning_rate=0.00005)
GANModel.compile(optimizer=opti , loss=wasserstein)#, metrics=['accuracy'])

set_trainable(discriminator, True)


In [ ]:
#plot_model(model, to_file=os.path.join(RUN_FOLDER ,'viz/model.png'), show_shapes = True, show_layer_names = True)
#plot_model(discriminator, to_file=os.path.join(RUN_FOLDER ,'viz/discriminator.png'), show_shapes = True, show_layer_names = True)
#plot_model(generator, to_file=os.path.join(RUN_FOLDER ,'viz/generator.png'), show_shapes = True, show_layer_names = True)

# Training of the GAN

In [ ]:
def train_normal(epochs, batch_size=128):
    # Save the Generator and discriminator models
    #save_model(generator, os.path.join(RUN_FOLDER, 'images/generator'))
    #save_model(discriminator, os.path.join(RUN_FOLDER, 'images/discriminator'))
    #save_model(GANModel, os.path.join(RUN_FOLDER, r"images/GANModel"))
    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = -np.ones((batch_size, 1))
    
    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------    
        # In a WGAN the Discriminator is trained multiple time
        for x in range(5):
            noise = np.random.normal(0, 1, (batch_size, z_dim))
            gen_imgs = generator.predict(noise)
            # Select a random half of images
            idx = np.random.randint(0, reshaped.shape[0], batch_size)
            imgs = reshaped[idx]
        
            # ---------------------
            #  Train the discriminator (real classified as ones and generated as zeros)
            # ---------------------
            d_loss_real = discriminator.train_on_batch(imgs, valid)
            d_loss_fake =  discriminator.train_on_batch(gen_imgs, fake)
            d_loss =  0.5 * (d_loss_real + d_loss_fake)
            
            # Weight clipping (Forster S.119)
            for l in discriminator.layers:
                weights = l.get_weights()
                weights = [np.clip(w, -clip_threshold, clip_threshold) for w in weights]
                l.set_weights(weights)

        # ---------------------
        #  Train Generator
        # ---------------------
        for x in range(1):
            g_loss = GANModel.train_on_batch(noise, valid)
        
        # ---------------------
        #  Save Losses for evaluation
        # ---------------------
        d = [d_loss, d_loss_real, d_loss_fake]
        d_losses.append(d)
        g_losses.append(g_loss)
        
        
        if (epoch % 100 == 0): 
            # Save an example
            fig=plt.figure(figsize=(64, 64))
            plt.imshow(gen_imgs[0, :, :, 0], cmap='gray')
            plt.axis('off')
            plt.savefig(os.path.join(RUN_FOLDER, "output/"+str(epoch)+".png"), format='png')
            plt.close()
            
            if (epoch % 1000 == 0):
                GANModel.save(os.path.join(RUN_FOLDER, 'weights/GANModel_'+str(epoch)+'_loss_'+str(g_loss)+'.h5'))
                discriminator.save(os.path.join(RUN_FOLDER, 'weights/discriminator_'+str(epoch)+'_loss_'+str(d_loss)+'.h5'))
                generator.save(os.path.join(RUN_FOLDER, 'weights/generator_'+str(epoch)+'_loss_'+str(g_loss)+'.h5'))
                
                # Continuiously save a plot with the new values to see the development of the loss
                fig = plt.figure()
                plt.plot([x[0] for x in d_losses], color='black', linewidth=0.25)
                plt.plot([x[1] for x in d_losses], color='green', linewidth=0.25)
                plt.plot([x[2] for x in d_losses], color='red', linewidth=0.25)
                plt.plot([g_losses], color='orange', linewidth=0.25)

                plt.xlabel('batch', fontsize=18)
                plt.ylabel('loss', fontsize=16)

                # plt.xlim(0, 2000)
                #plt.ylim(-50, 50)

                plt.savefig(os.path.join(RUN_FOLDER, "output/loss_chart.png"), format='png')
                plt.show()
                plt.close
                # Save the loss arrays
                np.save(os.path.join(RUN_FOLDER, "output/D_loss.npy"), d_losses)
                np.save(os.path.join(RUN_FOLDER, "output/G_loss.npy"), g_losses)
        # Plot the progress
        if (epoch % 10 == 0):
            print ("%d [D loss: (%.3f)(R %.3f, F %.3f)]  [G loss: %.3f] " % (epoch, d_loss, d_loss_real, d_loss_fake, g_loss))

In [ ]:
d_losses = []
g_losses = []

train_normal(10001, batch_size=32)

In [ ]:
fig = plt.figure()
plt.plot([x[0] for x in d_losses], color='black', linewidth=0.25)
plt.plot([x[1] for x in d_losses], color='green', linewidth=0.25)
plt.plot([x[2] for x in d_losses], color='red', linewidth=0.25)
plt.plot([x[0] for x in g_losses], color='orange', linewidth=0.25)

plt.xlabel('batch', fontsize=18)
plt.ylabel('loss', fontsize=16)

plt.xlim(0, 10000)
plt.ylim(-300000, 300000)

plt.savefig(os.path.join(RUN_FOLDER, "output/loss_chart3.png"), format='png')
plt.show()
plt.close

In [ ]:
# Quelle: https://stackoverflow.com/a/6537563
# Beep to tell training finished
import winsound
frequency = 300  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [ ]:
#d_losses = np.load(os.path.join(RUN_FOLDER, "images/D_loss.npy"))
#g_losses = np.load(os.path.join(RUN_FOLDER, "images/G_loss.npy"))